# <span style='color:darkred'> 3 Running an MD Simulation </span>

***

## <span style='color:darkred'> 3.1 Using the Advanced Research Computing (ARC) </span>


### <span style='color:darkred'> 3.1.1 Connect to the ARC </span>


*Note: if using Windows you will need to connect via MobaXterm as detailed in the setup instructions.*

Open a terminal and remotely connect to ARCUS using ssh (secure shell), using your username:

`% ssh username@oscgate.arc.ox.ac.uk`

When prompted, enter your password and hit Enter.
Now, you need to connect to the arcus-htc cluster, so in your command line type:

`% ssh arcus-htc`

You can now check the path to the directory you are at, by typing:

`% pwd`

It should be your home directory (/home/username). This is where you will work from.

### <span style='color:darkred'> 3.1.2 Obtain the required input files </span>

Now, you need to clone the repository where all the required input files for the simulation are stored. 

`% git clone https://github.com/bigginlab/OxCompBio-Datafiles.git`

Type:

`% ls`

You should see that a new directory has been generated, labelled `OxCompBio-Datafiles`

Move to this directory:

`% cd OxCompBio-Datafiles`

In there, if you type `ls` again, you should see four more subdirectories:

`data` Here, you will find the input files that are required to perform the MD simulation.

`setup` Here, you can perform all the steps to setup the protein system, as will be explained below.

`run` Here, you can perform the energy minimization and the MD production run.

`prerun` In this directory, you will find the data from an identical simulation that has already been perform. You can use these files for your analysis, in case you haven't managed for some reason to perform you own simulation successfully.

Now, go to the `setup` directory to start setting up the protein system:

`% cd setup`

***

## <span style='color:darkred'> 3.2 The GROMACS Molecular Dynamics engine </span>

[GROMACS](http://www.gromacs.org/) is a versatile package to perform molecular dynamics, i.e. simulate the Newtonian equations of motion for systems with hundreds to millions of particles. GROMACS is one of the many MD simulation packages, it is free to use and it is what we will be using for the purposes of this tutorial.  Note that this tutorial will only scratch the surface of all the powerful things GROMACS can do and there are many functionalities that we will not cover today.

### <span style='color:darkred'> 3.2.1 A brief overview of GROMACS file types </span>

Gromacs supports many different file types, here we give a brief overview of the different file types we will encounter during this tutorial.

- Coordinates files have the extension .gro and the default name is conf.gro.
- The topology file (default name topol.top) contains all the information about which atoms are bonded to which and what force-field parameters are applied etc.
- The trajectory files have the extension .xtc and .trr, the former does not contain velocity information and coordinates are held at a reduced precision, and so occupies less disk space. However you will need velocities if you want to continue a simulation.
- The .edr file contains the energy information from the trajectory.
- The .mdp file contains the information that was used to setup the actual simulation. Things related to temperature, pressure, how the electrostatics is calculated etc.
- The .tpr file is a binary file that contains all the information needed to perform the actual run (this allows gromacs to do lots of self-consistency checks to minimize user errors).

### <span style='color:darkred'> 3.2.2 Using GROMACS on ARCUS </span>

In order to use GROMACS on ARCUS, you need to load the already available module:

`% module load gpu/gromacs/2020.1`

By using the above command, the software becomes available for you to use, a bit like installing the software on the fly.

Check that it was successfully loaded, by typing:

`% gmx help commands`

This will print basic information for every built-in GROMACS command. If you need more detailed information on a command in particular, you can type:

`% gmx [command] -h`

***

##  <span style='color:darkred'>  3.3 Molecular Dynamics Simulation of Protein </span>

### <span style='color:darkred'> 3.3.1 Protein System Setup </span>


In this section, we will obtain our protein coordinates and perform some routine Molecular Dynamics calculations on them.

As previously discussed in notebook 2, we will use the HIV-1 protease structure (1HSG). It is a homodimer with two chains of 99 residues each. Even though you have already obtained the .pdb file of the protein from the [protein data bank](https://www.rcsb.org/), for this step it is probably easier to just copy it from the data directory to your working directory.

`% cp ../data/1hsg.pdb .` 

If you look at this file (using the graphical editors vi or nano for example, you should immediately see that it has two chains; A and B:

`% vi 1hsg.pdb`

or

`% nano 1hsg.pdb`

To quit the file, type:

- `% :q`, if you used `vi`

- `% Ctrl + x`, if you used `nano`


Now we need to prepare our protein for simulation. First of all we will extract only the protein coordinates from the pdb file into a new file called `protein.pdb`. To do this enter the following command:

`% grep ATOM 1hsg.pdb > protein.pdb`

Use your preferred text editor to open the protein.pdb file and see how it differs from the 1hsg.pdb file.

We now need to generate the parameter/topology file we need. This process will also make sure that all the hydrogens are added to our protein. 

`% gmx pdb2gmx -f protein.pdb -ignh -o protein.gro`

The `-ignh` flag will ignore any hydrogen atoms that might already be present in the .pdb file.

The program should run and present a list of force-fields from which to select. Select the AMBER99SB-ILDN force field which should be option 6 in the list followed by 1 to select the recommended TIP3P water. If all goes well this should generate several files:

1. topol.top 
2. topol_Protein_chain_A.itp 
3. topol_Protein_chain_B.itp 
3. posre_Protein_chain_A.itp
4. posre_Protein_chain_B.itp
5. protein.gro

Type:

`% ls`

to verify that all the above files have been created and are in your directory.\
Note that the protein has a net charge of +4e. You should see a line that says "Total charge
in system 4.000 e".

Before we can add water we need to define a box in which to put the protein and the water:

`% gmx editconf -f protein.gro -box 7 7 7 -c -o boxed.gro`

This puts the protein in the centre of the box that is 7 nm x 7 nm x 7 nm and creates the resulting file boxed.gro. As you may remember from the lecture, in this setup we will apply periodic boundary conditions, which means that the box is replicated indefinitely in all three directions. 

Next we need to add water to the system. This can be done by using the `gmx solvate` command, which will fill the box with water by repeatedly overlaying a small box of water into the system (216 molecules).

`% gmx solvate -cp boxed.gro -cs -o solvated.gro -p topol.top`

You may have noticed in some of the output generated that the total system charge is +4. In order for us to use an Ewald method to calculate the electrostatic interactions we need to have a neutral system overall. Therefore we will add counterions (chloride ions, in this case) using the option `-neutral` and enough ions to make the solution up to 150 mM (`-conc 0.15`). This is done by replacing random water molecules (SOL) with NA+ or CL- ions.

`% gmx grompp -c solvated.gro -p topol.top -f ../data/genion.mdp -o genion.tpr`\
`% gmx genion -s genion.tpr -conc 0.15 -neutral -pname NA -nname CL -o system.gro -p topol.top`

When prompted, enter the group that corresponds to SOL (should be 13 or thereabouts).

### <span style='color:darkred'> 3.3.2 Energy Minimization </span>

Before we can run the actual dynamics, we need to first minimize the energy of the system. This means that we need to allow the structure to relax so that the optimal geometry is obtained and there are no steric clashes between neighbouring atoms. Ideally you would minimize down until the forces were below a certain level (tolerance), but we will just give a quick burst of 200 steps here. Since we have finished setting up the system, we will now move to the `run` directory to perform our simulation from there:

`% cd ../run`

The `grompp` command will read the information of the system that we will provide (coordinates, topologies and simulation parameters) and will generate a .tpr run input file:

`% gmx grompp -c ../setup/system.gro -p ../setup/topol.top -f ../data/em.mdp -o em.tpr`

It is time now to invoke the `mdrun` command, which is the main engine within GROMACS which performs the MD simulation. We cannot run the simulation on the login nodes of ARC; these can only be used to prepare the system for the simulation. Therefore instead of typing the `mdrun` command -that will initiate the energy minimisation- directly on the command line, we will submit a script that will submit the job to the job scheduler. Copy this script to your working directory:

`% cp ../data/submit_em.sh .`

You can explore its contents and see that the last line in the file contains the `mdrun` command by typing: 

`% cat submit_em.sh`

The file contents will be printed on your terminal.\
Now submit it to the cluster queue.

`% sbatch submit_em.sh`

It will take a few minutes to run, depending on the waiting times of the queue. Check on the status of the run by typing:

`% squeue -u $USER`

Remember to replace `username` with your own username! It should print something like this:

`             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
     1311337        htc       EM       bioc1550 PD       0:00      1 (Priority) 
`

The `PD` in the fifth column denotes that the job is in the queue and has not started running yet. It will change to `R` once it starts running:

`             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
     1311337        htc       EM       bioc1550  R       1:26      1 arcus-htc-node110 
`

If the job has finished (or if it has failed), the above command will print nothing:

`JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)`

If you made a mistake and need to cancel this job, type:

`% scancel JOBID`

`JOBID` should be replaced by the ID of this particular job (printed in the first column; see above).

While the job is running, you can monitor its progress by typing: 

`% tail -n 12 em.log`

This will show you what step the simulation is at or if it has finished.\
Once the energy minimization has finished, we can examine it in terms of the potential energy versus the number of minimization steps:

`% gmx energy -s em.tpr -f em.edr -o em_potential_energy.xvg`

type in 10 when prompted (should correspond to potential energy from the list of options presented), then a zero to finish.

The output will be an `.xvg` file that will contain the system potential energy as a function of the minimization step.\
You can open the file with `vi` and observe whether the potential energy decreases during the energy minimization process, as would be expected:

`% vi em_potential_energy.xvg`

To quit the file, type:

`% :q`

### <span style='color:darkred'> 3.3.3 Production Run </span> 

At this stage, we would normally run a short simulation where the protein atoms are restrained while the water molecules
and ions are allowed to freely move around and equilibrate around the protein. This step would also allow the system to equilibrate at the desired temperature and pressure. For this tutorial however, we will skip this bit due to
limited time. Now finally let us perform some molecular dynamics:

`% gmx grompp -c em.gro -p ../setup/topol.top -f ../data/md.mdp -maxwarn 1 -o md.tpr`

`% cp ../data/submit_md.sh .`

`% sbatch submit_md.sh`

You can check the status of the job again as described previously.

At the moment it is set up to run for 1000 ps. This will take several minutes to complete depending on the waiting time in the queue - time for lunch! You don't have to wait for it to finish completely though, although now might be a good time for a break to allow at least some data to appear. The analysis can be done on the output files that will be generated or you can always use the output data from the simulation that we have already performed, found in the directory `../prerun/run` (This is 1000 ps simulation of the same system).


### <span style='color:darkred'> 3.3.4 MD trajectory modification </span>

During the simulation, the protein was free to rotate and translate and it is possible that it has diffused out of the box (remember that due to the periodic boundary conditions, the protein could appear to exit from one side of the box while entering from the opposite side). It will be easier to perform certain types of analysis later on, if we now put the protein back at the center of the box and then remove the rotational and translational motion. To this end, we need to modify the simulation trajectory `md.xtc' in three consecutive steps.

First, let's ensure that no atoms jump across the box and that the protein remains whole, using the `-pbc nojump` flag:

`% gmx trjconv -f md.xtc -s md.tpr -pbc nojump -o md_nojump.xtc`

When prompted, type `0` to include the entire system in the `md_nojump.xtc` trajectory.

In the next command, we will ensure that the center of mass of the protein is placed at the center of a compact box:

`% gmx trjconv -f md_nojump.xtc -s md.tpr -pbc mol -ur compact -center -o md_center.xtc`

When prompted, type `1` to center the protein in the box, and when you are prompted again, type `0` to include the entire system in the output trajectory `md_center.xtc`.

In the final step, we will remove the rotational and translation motion by fitting the protein to a reference structure, which in fact will be the starting structure of the simulation:

`% gmx trjconv -f md_center.xtc -s md.tpr -fit rot+trans -o md_fit.xtc`

When prompted, type `1` to select the protein for the least squares fit and then type `0` for the output.

The `.xtc` files are large in size and since we are only going to need the original trajectory (`md.xtc`) (never delete this!) and the final trajectory (`md_fit.xtc`), we can remove the intermediate trajectories that we generated:

`% rm md_nojump.xtc md_center.xtc`

Always be very cautious with the `rm` command!

### <span style='color:darkred'> 3.3.5 Obtain Temperature and Energy data </span>

After the end of the production run, it would be useful to obtain some properties that will give us an insight into our protein system.\
There are various so-called ensembles that are used for protein simulations - probably the most common is a system where the number of particles, the pressure and the temperature are held constant (NPT). This is usually achieved by employing algorithms that regulate the temperature and pressure, known as thermostats and barostats, respectively. Here, for the regulation of temperature, we use the velocity rescale thermostat, defined in the `md.mdp ` file, which couples the system to a "heat-bath" and ensures the reproduction of a correct kinetic ensemble.

Nevertheless, it is usually a good idea to check these as a function of time through the trajectory just to make sure nothing unexpected happened. First let us check the temperature of our simulation.

`% gmx energy -f md.edr -s md.tpr -o 1hsg_temperature.xvg`

The program will then present you with a large table of all the values recorded in the energy (.edr) file. We want to examine temperature so type 13, press enter and then 0 and press enter again. The program will then analyse the temperature and present some statistics of the analysis at the end.

Another set of properties that is quite useful to examine is the various energetic contributions to the energy. The total energy should be constant. but the various contributions can change and this can sometimes indicate something interesting or strange happening in your simulation. Let us look at some energetic properties of the simulation.

`% gmx energy -s md.tpr -f md.edr -o 1hsg_energies.xvg`

We shall select short-range lennard-jones (7), short range coulombic (9) and the potential energy (11). End your selection with a zero.

We will plot and explore the temperature and the energetic components that we obtained in the next section of the tutorial.


### <span style='color:darkred'> 3.3. File Transfer </span> 

As soon as the simulation is finished, you should go to your local terminal and transfer the files from the remote directory to your local directory.\

*Note: If your are using Windows and the MobaXterm, you will need to follow the instructions included in the `00_Setup` notebook.*

First, in your local terminal, go to the `OxCompBio/tutorials/MD` directory, i.e. the directory that the jupyter notebooks are located at.

The use `scp` to transfer the remote directory to your local directory:

`% scp -r username@oscgate.arc.ox.ac.uk:/home/username/OxCompBio-Datafiles/ .`

When prompted, enter your password. It might take a while for the files to be transfered. Once the transfer is finished, you can check that the files are indeed copied to the `OxCompBio-Datafiles` directory:

`% ls OxCompBio-Datafiles/`

Now you are ready to perform some types of analysis of the simulation trajectory!
